In this iPython notebook, we will featurize MOR ligand binding simulation by pairwise distances between the ligand and different receptor residues. We will then perform tICA and prospectively build an MSM. 

In [1]:
from PDB_Order_Fixer import PDB_Order_Fixer
import mdtraj as md
import os
import numpy as np
import h5py

import datetime
import glob
import copy
from functools import partial 
import operator
import time

import random 
import subprocess
from subprocess import Popen
import sys
from io_functions import *
from custom_clusterer import *
from custom_tica import *
from custom_featurizer import *
from pdb_editing import *
from analysis import *
from io_functions import *
#from topology_fixing import *
from subsampling import *
from conversions import *
from custom_msm import *
from grids import *

In [2]:
from detect_intermediates import *
from interpret_tICs import *

WHERE ARE WE?


In [3]:
from tutorial_feature_types import *
from get_variable_names import *
from tutorial_tica_config import *

tm6_tm3_residues
[A279, A165]
[65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 273, 274, 275, 276, 277, 278,

In [4]:
features_dir = "%s/ligand_protein_contacts" % (base)
print(features_dir)
if not os.path.exists(features_dir): os.makedirs(features_dir)
feature_residues_pkl = "%s/feature_residues.pkl" % features_dir
print(feature_residues_pkl)

/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts
/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/ligand_protein_contacts/feature_residues.pkl


In [5]:
ligand_residue = Residue(resSeq = 900, chain_id="J", res_name="LIG")

In [6]:
excluded_set = set(list(range(270, 292)) + list(range(239, 263)) + list(range(153, 170)) + list(range(170, 197)) + list(range(102, 116)) + list(range(80, 102)) + list(range(326, 350)))
included_residues = [res for res in contact_residues if res.resSeq not in excluded_set]
user_specified_contact_residue_pairs = [(ligand_residue, contact_residue) for contact_residue in included_residues]
print(user_specified_contact_residue_pairs)

[(J900, I225), (J900, I320), (J900, I208), (J900, I139), (J900, I213), (J900, I303), (J900, I236), (J900, I232), (J900, I69), (J900, I207), (J900, I118), (J900, I129), (J900, I201), (J900, I136), (J900, I307), (J900, I205), (J900, I197), (J900, I74), (J900, I67), (J900, I138), (J900, I124), (J900, I76), (J900, I301), (J900, I141), (J900, I292), (J900, I310), (J900, I311), (J900, I125), (J900, I305), (J900, I308), (J900, I309), (J900, I238), (J900, I324), (J900, I322), (J900, I318), (J900, I147), (J900, I315), (J900, I150), (J900, I140), (J900, I223), (J900, I77), (J900, I217), (J900, I224), (J900, I78), (J900, I132), (J900, I70), (J900, I126), (J900, I300), (J900, I200), (J900, I73), (J900, I221), (J900, I297), (J900, I145), (J900, I146), (J900, I117), (J900, I149), (J900, I321), (J900, I68), (J900, I313), (J900, I206), (J900, I135), (J900, I198), (J900, I210), (J900, I71), (J900, I128), (J900, I218), (J900, I296), (J900, I312), (J900, I314), (J900, I209), (J900, I120), (J900, I228), (

In [7]:
import pickle
with open(feature_residues_pkl, "rb") as f:
    feature_residues = pickle.load(f)
print(feature_residues[0])
print(feature_residues[0][0].__dict__)
print(feature_residues[0][1].__dict__)
tica_dir = "%s/tica_ligand_protein_contacts" % base


(J900, I304)
{'chain_name': None, 'res_name': 'LIG', 'ballosteros_weinstein': None, 'chain_id': 'J', 'resSeq': 900}
{'chain_name': None, 'res_name': 'ALA304', 'ballosteros_weinstein': None, 'chain_id': 'I', 'resSeq': 304}


In [8]:
projected_features_dir = "%s/phi_psi_chi2_allprot_projected.h5" % tica_dir
#plot_columns(tica_dir, projected_features_dir, titles = ["tIC%d" %j for j in range(1,6)], tICA = True, scale = 1.0, refcoords_file = None)

In [9]:
tic_components_dir = "%s/tic_components" % tica_dir
if not os.path.exists(tic_components_dir): os.makedirs(tic_components_dir)
projection_operator_dir = "%s/phi_psi_chi2_allprot_tica_coords.h5" % tica_dir
#interpret_tIC_components(projection_operator_dir, tic_components_dir, feature_residues_pkl, n_tica_components=5, percentile=95)


In [10]:
n_clusters = 20
clusterer_dir = "%s/clusterer_20clusters.h5" % tica_dir
n_samples=10
save_dir = "%s/clusters20_samples10" % tica_dir
sampling_method = "random"
clusters_map_file = "%s/clusters20_map.h5" % tica_dir

In [11]:
plot_all_tics_and_clusters(tica_dir, projected_features_dir, clusterer_dir, lag_time, label = "cluster_id", active_cluster_ids = range(0,20), intermediate_cluster_ids = [], inactive_cluster_ids = [])

loading "/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts/phi_psi_chi2_allprot_projected.h5"...
loading "/home/amir/Post_Process/GPCR/MOR/LIG_path/BU_path/tica_ligand_protein_contacts/clusterer_20clusters.h5"...
Looking at 5 tICS
Printed all tICA coords and all requested clusters


In [20]:
with open(feature_residues_pkl) as f:
    feature_residues = pickle.load(f)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [48]:
subset = crystal_structure.atom_slice(range(0,400))
subset.xyz
print(subset.xyz)
distances = md.compute_contacts(subset)
print(distances)


[[[-3.67750001 -2.00090003 -3.06229997]
  [-3.55780005 -1.92980003 -3.01830006]
  [-3.56550002 -1.78180003 -3.05369997]
  ..., 
  [-1.46140003 -0.32269999 -1.54359996]
  [-1.40170002 -0.40900001 -1.77069998]
  [-1.92920005 -0.43360001 -1.81420004]]]
(array([[ 0.29087129,  0.30480972,  0.50911838, ...,  0.31547278,
         0.29779878,  0.30303043]], dtype=float32), array([[ 0,  3],
       [ 0,  4],
       [ 0,  5],
       ..., 
       [47, 50],
       [47, 51],
       [48, 51]]))


In [20]:
distances0 = distances[0]

In [23]:
distances0 = np.nan_to_num(distances0)

In [24]:
distances0

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [25]:
np.where(distances0 > 0.0)

(array([], dtype=int64), array([], dtype=int64))